In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import random

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
text = open("data.txt", "r").read()
vocab = sorted(set(text))
vocab_size = len(vocab)

stoi = {ch:i for i, ch in enumerate(vocab)}
itos = {i:ch for ch, i in stoi.items()}

In [13]:
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.rt = nn.Linear(input_size + hidden_size, hidden_size)
    self.rtgate = nn.Sigmoid()
    self.zt = nn.Linear(input_size + hidden_size, hidden_size)
    self.ztgate = nn.Sigmoid()
    self.ht = nn.Linear(input_size + hidden_size, hidden_size)
    self.htact = nn.Tanh()
    self.o = nn.Linear(hidden_size, output_size)

  def init_hidden(self, device):
    return torch.zeros((1, self.hidden_size), device=device)

  def forward(self, input, hidden):
    inp = torch.cat((input, hidden), dim=1)
    rt = self.rtgate(self.rt(inp))
    zt = self.ztgate(self.zt(inp))
    hid_rt = hidden * rt
    candidate_inp = torch.cat((input, hid_rt), dim=1)
    ht = self.htact(self.ht(candidate_inp))
    new_hidden = (1 - zt) * ht + zt * hidden
    output = self.o(new_hidden)

    return output, new_hidden

In [18]:
input_size = output_size = vocab_size
hiddens = 96

gru = GRU(input_size, hiddens, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

In [19]:
def generate(start="", len_=20):
  chars = [stoi[start]]
  hidden = gru.init_hidden(device)
  for i in range(len_):
    inp = torch.tensor(chars[-1]).unsqueeze(0)
    inp = F.one_hot(inp, vocab_size).to(device)
    logits, hidden = gru(inp, hidden)
    probs = torch.softmax(logits, dim=1)
    ix = torch.multinomial(probs, 1).item()
    chars.append(ix)
  return "".join(itos[ix] for ix in chars)

In [20]:
from tqdm import tqdm

In [21]:
context_window = 24
chunk_size = 2000
epochs = 50

for epoch in range(epochs):
  ix = torch.randint(0, len(text)-chunk_size-2, size=(1,)).item()
  for i in tqdm(range(ix, ix+chunk_size-context_window)):
    chunk = text[i:i+context_window]
    target = text[i+1:i+1+context_window]
    chunk = torch.tensor([stoi[ch] for ch in chunk])
    target = [stoi[ch] for ch in target]
    chunk = F.one_hot(chunk, vocab_size).to(device)
    target = torch.tensor(target).to(device)

    hidden = gru.init_hidden(device)
    total_loss = 0
    for x in range(chunk.shape[0]):
      inp = chunk[x].unsqueeze(0)
      tg = target[x].unsqueeze(0)
      logits, hidden = gru(inp, hidden)
      loss = loss_fn(logits, tg)
      total_loss += loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
  if (epoch+1) % 50 == 0:  # bruh
    print(total_loss.item())
    print(generate(random.choice(vocab), 300))

100%|██████████| 1976/1976 [01:02<00:00, 31.54it/s]


49.757843017578125
мелими нащим изи чём нек за, за в ду и ещё бы не завернял ял ни валерна валаж.
Я вет за ращищи вид и наветпяля от моёсо вдмикизна бок в ото!
Вы, чё денова. Броволли и не вадми- не ещёоти с смото межкимо- из купить? прогледя я в порокам пимой из нупадибо могли омовя навибо грася терпаящий прий нискост


In [23]:
torch.save(gru.state_dict(), "/content/gru_params.pth")

In [52]:
print(generate("Д", 1_000))

Да, коконкогодить, мочканом дирововня и порока к не взгли. Я намилк и видняной вез но! Я занил Джону ве и и настолми комнуе. Дорупалошким ос ещё ил и отющибуборащи.
Полета дола и пароком.
Я вар.
Я градиз споорносто очкосчавачеравидяты, чтобот. Потой от стой, вы ну, забиравива протновило в вернимокну межния стоу верпока.
Нока мне. Того граят от. Подой, и визбудее каком и простоиби сегода.. мне и из бартела, вои и идно тобобкя они отдыпивастьет, !
Я и из убирника я вадот. Лооящия  вожк прощенома вся писье запяровив скоДупорока и вернула каки на оты сайдя ули мне гори в мела
ми отой каком идилизмолое вст соба и До, запикий дасчу, не звисниз нате мёнча, из на пожил видяибийда визнивернавили в дудо вабот, к оновилько мрисли:
У нам хотолжиошь отдиби да мой идя он ношо. ли васпраглечто можно комне обятьке забанная расотне, звароки, монянна ми кечкомолжазбили дом, исти в мниме могом доми-тлоятьсти и кононь л жиное стерупимеми отрющая саровенила видо ни к из вись тыбивший и ведсотот вваероко бо

### Well, looks like Glorps language, transfers style. Well done!
<img src="https://media.tenor.com/sEiYXWmf1W8AAAAi/glorp-alien.gif">